In [3]:
from ipywidgets import FloatSlider, ValueWidget
import ipywidgets as widgets

from sidepanel import SidePanel

from regulus.utils import io
from regulus.topo import * 
from regulus.alg import *
from regulus.measures import *
from regulus.models import *
from regulus.tree import *

from ipyregulus import TreeWidget, TreeView, DetailsView, DataWidget
from ipyregulus.filters import * 

In [4]:
data = io.load('data/gauss4.p')
# data = io.load('data/simulations.p')

In [5]:
data.add_attr('linear', node_model(linear_model))
data.add_attr('fitness', fitness)
data.add_attr('relative_fitness', relative_fitness)

data.tree.add_attr('parent_fitness', parent_fitness)
data.tree.add_attr('child_fitness', child_fitness)
data.tree.add_attr('size', node_size)
data.tree.add_attr('rel_size', node_relative_size)
data.tree.add_attr('span', node_span)

In [6]:
from ipywidgets import Output
debug = Output(title='debug')
debug

Output()

### Helper functions

#### view

In [7]:
def update_view(view, f):
#     @debug.capture()
    def _inner():
        print('update_view')
        view.show = filter_tree(view.tree, f)
    return _inner 

In [8]:
def view(data, attr='span', func=lambda x, v: v <= x, title=''):
    v = TreeView(data, attr=attr)
    f = AttrFilter(attr=attr, func=func)
    m = Monitor(f, func=update_view(v, f))
    p = SidePanel(title=title)
    with p:
        display(v,f) 
    return v, f, p

#### reduce

In [9]:
def reduce_view(widget, tree, f):
    @debug.capture()
    def _inner():
        widget.ref = reduce_tree(tree, filter=f)
    return _inner

In [10]:
def reduce(data, attr='span', func=lambda x, v: v <= x, title=''):
    v = TreeView(data.tree, attr=attr)
    f = AttrFilter(attr=attr, func=func)
    m = Monitor(f, func=reduce_view(v, data.tree, f))
    p = SidePanel(title=title)
    with p:
        display(v,f) 
    return v, f, p

### Initial tree

In [11]:
tw = TreeWidget(data.tree)

In [12]:
v1, f1, p1 = view(tw, title='Initial Tree')

In [23]:
v2, f2, p2 = view(tw, attr='fitness', title='fitness')

### Reduce tree

In [14]:
v2, r2, p2 = reduce(tw, attr='span', title='reduce')

In [15]:
v2.attr = 'parent_fitness'

### fitness

In [16]:
v2

TreeView(field='parent_fitness', tree_model=TreeWidget(attrs={'span': {0: 0.5392374574080602, 1: 0.00980789566…

#### test

In [17]:
tw2 = TreeWidget(data.tree)

In [18]:
f = AttrFilter(attr='span', func=lambda x, v: v <= x)

In [19]:
@debug.capture()
def test():
    print('test')
    tw2.ref = reduce_tree(tw.tree, filter=f)

In [20]:
m = Monitor(f, func=test)

In [21]:
test_view = TreeView(tw2, attr='span') 

In [22]:
p = SidePanel(title='Test')
with p:
    display(test_view, f)

In [23]:
fitness = view(tw2, attr='fitness', title='fitness')

In [24]:
parent_fitness = view(tw2, attr='parent_fitness', title='parent_fitness')

In [25]:
f_view = TreeView(tw2, attr='fitness')

In [24]:
f_view

NameError: name 'f_view' is not defined

In [27]:
test_view

TreeView(field='span', tree_model=TreeWidget(attrs={'span': {0: 0.5392374574080602, 1: 0.00980789566443424, 2:…

In [28]:
v1

TreeView(field='span', tree_model=TreeWidget(attrs={'span': {0: 0.5392374574080602, 1: 0.00980789566443424, 2:…